In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [9]:
df = pd.read_csv('exchange-rate-twi.csv')
df = df.set_index('Month')
df.head()

,Exchange Rate TWI
Month,
1,100.0
2,99.6
3,99.4
4,99.1
5,99.2


In [10]:
train_len = 212
train = df[:train_len]
test = df[train_len:]

### Simple Average Method

In [11]:
y_hat_avg = test.copy()
y_hat_avg['avg_forecast'] = train['Exchange Rate TWI'].mean()
y_hat_avg['avg_forecast']

Month
213    88.325472
214    88.325472
215    88.325472
216    88.325472
217    88.325472
         ...    
300    88.325472
301    88.325472
302    88.325472
303    88.325472
304    88.325472
Name: avg_forecast, Length: 92, dtype: float64

In [17]:
##RMSE and mape
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(np.mean(mean_squared_error(test['Exchange Rate TWI'],y_hat_avg['avg_forecast']))).round(2)
mape = np.round(np.mean((np.abs(test['Exchange Rate TWI']-y_hat_avg['avg_forecast'])/test['Exchange Rate TWI'])*100),2)
results = pd.DataFrame({'Method':['Simple Average'],
                       'RMSE':[rmse],
                       'MAPE':[mape]})
results

,Method,RMSE,MAPE
0,Simple Average,32.42,58.12


### Naive Forecast

In [14]:
y_hat_naive = test.copy()
y_hat_naive['naive_forecast'] = train['Exchange Rate TWI'][train_len]
y_hat_naive['naive_forecast']

Month
213    52.0
214    52.0
215    52.0
216    52.0
217    52.0
       ... 
300    52.0
301    52.0
302    52.0
303    52.0
304    52.0
Name: naive_forecast, Length: 92, dtype: float64

In [19]:
## RMSE and MAPE value of Naive Forecast
rmse = np.sqrt(np.mean(mean_squared_error(test['Exchange Rate TWI'],y_hat_naive['naive_forecast']))).round(2)
mape = np.round(np.mean((np.abs(test['Exchange Rate TWI']-y_hat_naive['naive_forecast'])/test['Exchange Rate TWI'])*100),2)
results.loc[len(results)] = ['Naive Method',rmse,mape]

In [20]:
results

,Method,RMSE,MAPE
0,Simple Average,32.42,58.12
1,Naive Method,5.98,8.41


In [22]:
### Accuracy of naive forecasting 
100- mape

91.59

### Simple Moving Average window

In [23]:
y_hat_sma = df.copy()
y_hat_sma['sma_forecast'] = df['Exchange Rate TWI'].rolling(12).mean()
y_hat_sma['sma_forecast'][train_len:] = y_hat_sma['sma_forecast'][train_len-1]

In [29]:
## rmse and mape
rmse = np.sqrt(np.mean(mean_squared_error(test['Exchange Rate TWI'],y_hat_sma['sma_forecast'][train_len:]))).round(2)
mape = np.round(np.mean((np.abs(test['Exchange Rate TWI']-y_hat_sma['sma_forecast'][train_len:])/test['Exchange Rate TWI'])*100),2)
results.loc[len(results)] = ['Simple Moving Average',rmse,mape]
results

,Method,RMSE,MAPE
0,Simple Average,32.42,58.12
1,Naive Method,5.98,8.41
2,Simple Moving Average,4.57,6.89


In [30]:
results

,Method,RMSE,MAPE
0,Simple Average,32.42,58.12
1,Naive Method,5.98,8.41
2,Simple Moving Average,4.57,6.89


In [31]:
y_hat_sma['sma_forecast_6'] = df['Exchange Rate TWI'].rolling(6).mean()
y_hat_sma['sma_forecast_6'][train_len:] = y_hat_sma['sma_forecast_6'][train_len-1]

In [32]:
## rmse and mape
rmse = np.sqrt(np.mean(mean_squared_error(test['Exchange Rate TWI'],y_hat_sma['sma_forecast_6'][train_len:]))).round(2)
mape = np.round(np.mean((np.abs(test['Exchange Rate TWI']-y_hat_sma['sma_forecast_6'][train_len:])/test['Exchange Rate TWI'])*100),2)
results.loc[len(results)] = ['Simple Moving Average_6',rmse,mape]
results

,Method,RMSE,MAPE
0,Simple Average,32.42,58.12
1,Naive Method,5.98,8.41
2,Simple Moving Average,4.57,6.89
3,Simple Moving Average_6,4.58,6.90


In [33]:
y_hat_sma['sma_forecast_3'] = df['Exchange Rate TWI'].rolling(3).mean()
y_hat_sma['sma_forecast_3'][train_len:] = y_hat_sma['sma_forecast_3'][train_len-1]

In [34]:
## rmse and mape
rmse = np.sqrt(np.mean(mean_squared_error(test['Exchange Rate TWI'],y_hat_sma['sma_forecast_3'][train_len:]))).round(2)
mape = np.round(np.mean((np.abs(test['Exchange Rate TWI']-y_hat_sma['sma_forecast_3'][train_len:])/test['Exchange Rate TWI'])*100),2)
results.loc[len(results)] = ['Simple Moving Average_3',rmse,mape]
results

,Method,RMSE,MAPE
0,Simple Average,32.42,58.12
1,Naive Method,5.98,8.41
2,Simple Moving Average,4.57,6.89
3,Simple Moving Average_6,4.58,6.90
4,Simple Moving Average_3,5.10,7.32


### ADF test

In [38]:
from statsmodels.tsa.stattools import adfuller
adf_test = adfuller(df['Exchange Rate TWI'])
print('ADF Statistic: %0.2f' %adf_test[0])
print('Critical value @ 0.05: %0.2f' %adf_test[4]['5%'])
print('p-value:%0.2f' %adf_test[1])

ADF Statistic: -0.81
Critical value @ 0.05: -2.87
p-value:0.82


### KPSS test

In [40]:
from statsmodels.tsa.stattools import kpss
kpss_test = kpss(df['Exchange Rate TWI'])
print('KPSS Statistic: %.3f' % kpss_test[0])
print('Critical Value: @ 0.05: %0.2f' %kpss_test[3]['5%'])
print('p-value: %f' %kpss_test[1])

KPSS Statistic: 2.614
Critical Value: @ 0.05: 0.46
p-value: 0.010000
